In [139]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os

In [135]:
keyG = 'fad78733-31a0-4ea7-8823-ba815b578899'
keyNYT = ''

guardian = requests.get(
    'https://content.guardianapis.com/search?api-key=' + keyG +
    '&from-date=2022-02-01' +
    '&type=article' +
    '&page=1' +
    '&tag=world/ukraine' +
    '&order-by=newest' +
    '&show-fields=body' +
    '&page-size=200')

json_guardian = guardian.json()
#print(json.dumps(json_data, indent=2))
#print(json_data)

In [147]:
urls = []
titles = []
bodies = []
tempbody = ""
dates = []

for i in range (1,200):
    urls.append(json_guardian['response']['results'][i]['webUrl'])
    titles.append(json_guardian['response']['results'][i]['webTitle'])
    dates.append(json_guardian['response']['results'][i]['webPublicationDate'] )
    soup = BeautifulSoup(json_guardian['response']['results'][i]['fields']['body'], "html.parser")
    tempbody = soup.find_all("p", class_='dcr-xry7m2').get_text()
    bodies.append(tempbody)

dict_data = {"URL": urls, "Date": dates, "Title": titles, "Text": bodies}
data = pd.DataFrame(dict_data)

os.makedirs(os.getcwd() + "/data", exist_ok=True)
data.to_csv("data/guardian.csv", index=True, header=True)
data.head(25)

AttributeError: ResultSet object has no attribute 'get_text'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
txt_body = BeautifulSoup(body, "html.parser").get_text().replace("  Sign up to First Edition, our free daily newsletter – every weekday morning at 7am BST  ","")
txt_title = BeautifulSoup(title, "html.parser").get_text()

print(txt_title)
print(txt_body)